In [6]:
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

# define the transform to convert PIL images to tensors
transform = transforms.Compose([transforms.ToTensor()])

# load the MNIST dataset with the defined transform
mnist = MNIST(root='.', download=True, transform=transform)

# prepare the data for training
dataloader = DataLoader(mnist, batch_size=32, shuffle=True)

# define the self-supervised model
class Autoencoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28*28, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU()
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 28*28),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = Autoencoder()
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.MSELoss()

# train the model
for epoch in range(10):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        output = model(img)
        loss = criterion(output, img)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.015814226120710373
Epoch 2, Loss: 0.011125342920422554
Epoch 3, Loss: 0.010279752314090729
Epoch 4, Loss: 0.007550423964858055
Epoch 5, Loss: 0.008284318260848522
Epoch 6, Loss: 0.007360922638326883
Epoch 7, Loss: 0.0058225770480930805
Epoch 8, Loss: 0.006143888458609581
Epoch 9, Loss: 0.005919524002820253
Epoch 10, Loss: 0.005869313143193722


In [7]:
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

# define the transform to convert PIL images to tensors
transform = transforms.Compose([transforms.ToTensor()])

# load the MNIST dataset with the defined transform
mnist = MNIST(root='.', download=True, transform=transform)

# prepare the data for training
dataloader = DataLoader(mnist, batch_size=32, shuffle=True)

# define the self-supervised model
class Generator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 28*28),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.model(x)
        return x

model = Generator()
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.BCELoss()

# train the model
for epoch in range(10):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        noise = torch.randn(img.size(0), 64)
        output = model(noise)
        loss = criterion(output, img)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# generate new images
noise = torch.randn(16, 64)
gen_imgs = model(noise)
gen_imgs = gen_imgs.view(gen_imgs.size(0), 1, 28, 28)


Epoch 1, Loss: 0.2736301124095917
Epoch 2, Loss: 0.26214495301246643
Epoch 3, Loss: 0.257989764213562
Epoch 4, Loss: 0.2658120095729828
Epoch 5, Loss: 0.25503048300743103
Epoch 6, Loss: 0.27823132276535034
Epoch 7, Loss: 0.26343193650245667
Epoch 8, Loss: 0.2714965343475342
Epoch 9, Loss: 0.26465362310409546
Epoch 10, Loss: 0.27469226717948914


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define the autoencoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=2, stride=2),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Define the classifier
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(3072, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Load the data
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

# Define the loss function and optimizer
autoencoder = Autoencoder()
classifier = Classifier()
criterion = nn.CrossEntropyLoss()
criterion_classifier = nn.CrossEntropyLoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Train the autoencoder
for epoch in range(10):
    for data in train_loader:
        img, label = data
        optimizer.zero_grad()
        output = autoencoder(img)
        pred = classifier(output)
        loss = criterion(output, img) + criterion_classifier(pred, label)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch}: Loss: {loss.item()}")

Files already downloaded and verified
Files already downloaded and verified
Epoch 0: Loss: 3.86711049079895
Epoch 1: Loss: 3.6304707527160645
Epoch 2: Loss: 3.750314712524414
Epoch 3: Loss: 3.6846940517425537
Epoch 4: Loss: 3.772843837738037
Epoch 5: Loss: 3.9253616333007812
Epoch 6: Loss: 3.7198665142059326
Epoch 7: Loss: 3.631049156188965
Epoch 8: Loss: 3.657764434814453
Epoch 9: Loss: 3.7344417572021484


In [52]:
for data, label in train_loader:
    break

In [53]:
i1 = data[1].view(1, 3, 32, 32)
o1 = autoencoder.encoder(i1)
d1 = autoencoder.decoder(o1)
d1.shape

torch.Size([1, 3, 32, 32])

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

# Define your model architecture
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = nn.MaxPool2d(2)(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = nn.MaxPool2d(2)(x)
        x = x.view(-1, 128 * 8 * 8)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

# Initialize model, criterion, and optimizer
model = ImageClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Load your data
transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.RandomRotation(10), transforms.RandomCrop(32, padding=4), transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / (i + 1)))

# # Test the model
# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data
#         outputs = model(images)
#         _, predicted = torch.max(outputs)

Files already downloaded and verified
Files already downloaded and verified
Epoch 1 loss: 2.231
Epoch 2 loss: 2.023
Epoch 3 loss: 1.937
Epoch 4 loss: 1.829
Epoch 5 loss: 1.731
Epoch 6 loss: 1.654
Epoch 7 loss: 1.595
Epoch 8 loss: 1.544
Epoch 9 loss: 1.511
Epoch 10 loss: 1.488
